In [ ]:
from pathlib import Path
from tqdm import tqdm
import torch
from torch import nn
import torch.nn.functional as F
from torchvision import transforms
from PIL import Image
import pandas as pd
import os
import numpy as np
import sys
import os
# !{sys.executable} -m pip install loguru

parent_dir = os.path.dirname(os.path.abspath("__file__"))  # If your notebook is in VisionModels, this will be its path
sys.path.append(parent_dir)

from fewshot.feature_extractors import load_feature_extractor, FEATURE_EXTRACTORS


In [ ]:
# Set feature extractor here!
FEATURE_EXTRACTOR = "phikon"
# FEATURE_EXTRACTOR = "vit"
# FEATURE_EXTRACTOR = "resnet50"

In [ ]:
IMAGE_DIRS = {
    "CRC100K": Path("./data/CRC-VAL-HE-7K-png"),
    "MSSI": Path("./data/MSSI"),
    "PCAM": Path("./data/PCam/full_imgs"),
}

# Set feature extractor here!
FEATURE_EXTRACTOR = "phikon"
# FEATURE_EXTRACTOR = "vit"
# FEATURE_EXTRACTOR = "resnet50"

FEATURE_DIM = 768 if FEATURE_EXTRACTOR not in {"resnet50", "retccl", "bt", "swav", "mocov2"} else 2048

print(f"Available feature extractors: {', '.join(FEATURE_EXTRACTORS.keys())}")
print(f"Using feature extractor: {FEATURE_EXTRACTOR}")

backend = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
feature_extractor = load_feature_extractor(FEATURE_EXTRACTOR)
feature_extractor = feature_extractor.to(backend)
feature_extractor.eval()

to_tensor = transforms.ToTensor()

In [ ]:
def create_img_embeddings(dataset_name):
    image_paths = list(IMAGE_DIRS[dataset_name].rglob("**/*.png"))

    with torch.inference_mode():
        features = {}
        for img_path in tqdm(image_paths, desc="Extracting features", leave=False):
            img = Image.open(img_path)
            img = to_tensor(img).unsqueeze(0).to(backend)
            # features[img_path.name] = feature_extractor(img).squeeze().detach().cpu().numpy()
            features[img_path] = feature_extractor(img).squeeze().detach().cpu().numpy()

    vector_len = len(next(iter(features.values())))
    struct = np.dtype([("name", "U400"), ("vector", "f4", (vector_len,))])
    feat_array = np.array(list(features.items()), dtype=struct)

    if not os.path.exists("features"):
        os.makedirs("features") 
    save_path = f"features/{FEATURE_EXTRACTOR}_{dataset_name}_features.npy"
    np.save(save_path, feat_array)

    print(f"Saved features of shape {feat_array.shape} to {save_path}")



In [ ]:
def load_and_rename_embeddings(file_path):
    feat_array = np.load(file_path, allow_pickle=True)

    if feat_array.dtype.kind != 'V':
        vector_len = feat_array[0][1].shape[0]
        struct = np.dtype([('name', 'U10'), ('vector', 'f4', (vector_len,))])
        feat_array = np.array([(name, vec) for name, vec in feat_array], dtype=struct)

    for item in feat_array:
        new_name = item["name"].replace("0-PCAM", "NORM-PCAM")
        new_name = new_name.replace("1-PCAM", "TUM-PCAM")
        item["name"] = new_name

    np.save(file_path, feat_array)

load_and_rename_embeddings("./VisionModels/fewshot-histo/features/phikon_PCAM_features.npy")

In [ ]:
#create_img_embeddings("CRC100K")
#create_img_embeddings("MSSI")

In [ ]:
backend = torch.device("cpu")
feature_extractor.to(backend)
create_img_embeddings("PCAM")